# Inverse dynamics optimization on real data
- This notebook is dedicated to testing the optimization process for inverse dynamics 
- Starting from a rough solution estimated by inverse kinematics
- No consideration of 2 projections is considered here (at least yet!)

:+1: Process either real or simulated data

In [ ]:
import time 

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pinocchio as pin
from scipy.optimize import least_squares
from scipy.special import huber
from projectyl.utils.properties import ELBOW, SHOULDER, WRIST
from projectyl.utils.arm import plot_optimization_curves

from projectyl.dynamics.inverse_kinematics import build_arm_model
from projectyl.utils.arm import interactive_replay_sequence
from projectyl.utils.io import Dump

from projectyl.dynamics.inverse_dynamics import objective
%load_ext autoreload
%autoreload 2
global_params = {}
build_arm_model(global_params, headless=False)
arm_robot = global_params["arm"]
viz = global_params["viz"]
SIMULATION = "simulation"
REAL = "real"

# :warning: PICK **REAL** or **SIMULATION**

In [ ]:
# PICK UP THE MODE HERE
mode = SIMULATION
# mode = REAL
assert mode in [SIMULATION, REAL]

# REAL DATA
### Load data


To load data, you need a preprocessed video file. Assume IK has been performed first.

`python scripts/batch_video_processing.py -i "data/*9_*.mp4" -o __out -A ik --headless`

In [ ]:
if mode == REAL:
    data = Dump.load_pickle("../__out/0009_weight_lifting_main_camera_30fps/coarse_ik.pkl")

    # INITIAL STATE LIST
    q = np.array(data["q"])
    T = len(q)

    # 3D points
    p = ([np.array([el.translation for el in data["3dpoints"][member]]).T for member in [SHOULDER, ELBOW, WRIST]])
    gt_p_full = np.concatenate(p).T
    gt_p_full.shape #T, 9

    # initialization from IK estimation
    initial_full = np.concatenate([q, np.zeros((T, 2 * arm_robot.model.nv))], axis=1)
    initial_full.shape
    DT = 1/30. # 30fps

# SIMULATION


In [ ]:
if mode == SIMULATION:
    q = arm_robot.q0.copy()
    vq = np.zeros(arm_robot.model.nv)
    aq = 5 * np.random.rand(arm_robot.model.nv)
    viz.display(q)
    DT = 1e-2

    T = 30

    rec_gt_sol = []
    rec_gt_p = []

    shoulder_frame_id = arm_robot.model.getFrameId(SHOULDER)
    elbow_frame_id = arm_robot.model.getFrameId(ELBOW)
    wrist_frame_id = arm_robot.model.getFrameId(WRIST)

    for _ in range(T):
        # Iterative forward dynamics
        vq += aq * DT
        q = pin.integrate(arm_robot.model, q, vq * DT)
        tauq = pin.rnea(arm_robot.model, arm_robot.data, q, vq, aq)
        
        # Retrieve 3D points (forward kinematics)
        pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)
        shoulder_p = arm_robot.data.oMf[shoulder_frame_id].translation
        elbow_p = arm_robot.data.oMf[elbow_frame_id].translation
        end_effector_p = arm_robot.data.oMf[wrist_frame_id].translation

        viz.display(q)
        # Store ground truth "solutions" (q, vq, tau)
        xuc = np.concatenate([q, vq, tauq])
        rec_gt_sol.append(xuc)
        # Store ground truth data (p)
        p = np.concatenate([shoulder_p, elbow_p, end_effector_p])
        rec_gt_p.append(p)
        time.sleep(1e-2)

    q.shape # 4 +1 DOF elbow
    vq.shape # 4 =  3(axis angle 3D ~  log map of quaternion 4D ) + 1 DOF elbow
    tauq.shape # 4 = 3 + 1 
    xuc.shape # 5 + 4 + 4 = 13  = position + velocity + torque

    gt_sol = np.vstack(rec_gt_sol)
    gt_p_full = np.vstack(rec_gt_p)
    gt_sol.shape, gt_p_full.shape # T=180 step, 13 "xuc" and 9 3D positions
    # sol = solution...
    # p = position = oberserved data...


    q = gt_sol[:, :arm_robot.model.nq]

    initial_full = np.concatenate([q, np.zeros((T, 2 * arm_robot.model.nv))], axis=1)

### Define the whole optimizer
> TODO: Move to a python file instead of notebook

### Define trimming
To process just a piece of the sequence

In [ ]:
### DEFINE TRIMMING
trim = 30
assert trim <= T
initial= initial_full[:trim]
gt_p = gt_p_full[:trim]
assert gt_p.shape[1] ==9
assert initial.shape[1] ==13

### OPTIMIZE

In [ ]:
sol = least_squares(objective, initial.flatten(), args=(gt_p, trim, DT, arm_robot)).x
res = sol.reshape(trim, -1)

### PLOT CURVES

In [ ]:
plot_optimization_curves([
    (initial, "[gt]", "--"),
    ],
    title="Initial guess"
)
plot_optimization_curves([
    (res, "[estimation]", "-"),
    ],
    title="Optimizer prediction"
)

In [ ]:
plot_optimization_curves([
    (res, "[estimation]", "-"),
    ],
    title="Optimizer prediction",
    mode="q"
)

### VISUALIZE INTERACTIVELY

In [ ]:
interactive_replay_sequence(
    {
        "estimated": res[:, :arm_robot.model.nq],
        "groundtruth": initial[:, :arm_robot.model.nq],
    },
    viz
)